In [ ]:
from lxml import html
import requests
from bs4 import BeautifulSoup

import pandas as pd
import time

In [ ]:
def get_page(page_number):
    list_url = []
    try:
        page = requests.get('https://www.allabolag.se/where/stockholm/xb/AB/xl?page=' + str(page_number))
        soup = BeautifulSoup(page.content, 'html.parser')
        res = soup.find_all('h2', class_="search-results__item__title") 
        list_url = [x.find('a').get('href') for x in res]

    except:
        pass
    return list_url

In [ ]:
def get_company_info(url):
    
    c_dict = {}
    df = pd.DataFrame()
    
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        company_table = soup.find('table', class_="company-table") 
        company_name = url.split('/')[-1].replace('-', ' ') 

        if company_table:
            data_rows = company_table.find_all('tr')
            years = company_table.find_all('th', class_='data-pager__page')

            for row in data_rows:

                if row.th and row.th.text:
                    name = row.th.text.strip()
                    if name == 'Bokslut & nyckeltal':
                        continue
                    values = row.find_all('td')

                    data=[]
                    company_names=[]
                    for value in values:    
                        clean_value = value.text.strip().replace('%', '').replace(',','.')
                        if clean_value:
                            try:
                                data.append(float(clean_value))
                                company_names.append(company_name)
                            except ValueError:
                                data.append(0)

                    c_dict[name] = data
                    c_dict['company_name'] = company_names

            years_list = []        
            for row in years:
                if row.text:
                    year = row.text.strip().split('-')[0]
                    years_list.append(year)
            c_dict['year'] = years_list


            df = pd.DataFrame(data = c_dict).set_index('company_name')
    except:
        pass

    return df         

In [ ]:
def scrape_pages(n):
    data = pd.DataFrame()
    
    for i in range(1,n+1):
        urls = get_page(i)
        
        for url in (urls):
            try: 
                data = data.append(get_company_info(url))
                time.sleep(2)
            except ValueError:
                print 'no info'
        time.sleep(2)
        
    return data

In [ ]:
all_pages = scrape_pages(400)
all_pages.to_csv('./all_companies_info.csv', encoding='utf-8')